In [ ]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
tensor_A = torch.rand(7,7)
tensor_B = torch.rand(1,7)

In [ ]:
torch.matmul(tensor_A, tensor_B.T)

In [ ]:
torch.manual_seed(seed=0)
random_tensor_A = torch.rand(3, 4)

torch.random.manual_seed(seed=0)
random_tensor_B = torch.rand(3, 4)



In [ ]:
random_tensor_A, random_tensor_B

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
torch.cuda.manual_seed(seed=1234)
random_tensor_C = torch.randn(2,3).to(device)

torch.cuda.manual_seed(seed=1234)
random_tensor_D = torch.randn(2,3).to(device)

random_tensor_C, random_tensor_D

In [ ]:
out = torch.matmul(random_tensor_C, random_tensor_D.T)

print(out)

In [ ]:
print(torch.max(out))
print(torch.min(out))
print(torch.argmax(out))
print(torch.argmin(out))

In [ ]:
tensor_multi = torch.rand(1,1,1,10)
print(tensor_multi, tensor_multi.shape)

In [ ]:
tensor_E = torch.squeeze(tensor_multi)
tensor_E, tensor_E.shape

### WorkFlow Fundamentals

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu';
device

In [ ]:
wt = 0.3
bias = 0.9

X = torch.range(0,1,0.002).unsqueeze(dim=1)
y = wt * X + bias

X.shape, y.shape

In [ ]:
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]


In [ ]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=0.7, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=0.7, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=1, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});

In [ ]:
plot_predictions();

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, dtype=torch.float), requires_grad=True)
        self.bias = nn.Parameter(torch.randn(1, dtype=torch.float), requires_grad=True)

    def forward(self, x:torch.tensor) -> torch.tensor:
        return self.weights * x + self.bias


In [ ]:
mdl = LinearRegression().to(device)

In [ ]:
list(mdl.parameters())

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params= mdl.parameters(), lr=0.01)

In [ ]:
torch.manual_seed(42)
EPOCHS = 800

X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

for epoch in range(EPOCHS):
    mdl.train()

    y_pred = mdl(X_train)

    loss = loss_fn(y_pred, y_train)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    mdl.eval()

    with torch.inference_mode():
        test_pred = mdl(X_test)

        test_loss = loss_fn(test_pred, y_test)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")


In [ ]:
print(f" Predicted weights and biases --> {mdl.state_dict()}\n")

print(f" OG weights & bias {wt} and {bias}")

In [ ]:
plot_predictions(predictions= test_pred.cpu())

In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "00_linregres_model.pth"
MODEL_SAVE_PATH =  MODEL_PATH / MODEL_NAME

torch.save(obj=mdl.state_dict(), f=MODEL_SAVE_PATH)
